In [54]:
import pandas as pd
import folium


Raw Dataframe (2020년 8월 위성 데이터 사용)

In [55]:
df = pd.read_csv(r"../Data/2020-08.csv", encoding="EUC-KR")
#C:\Users\HOME\Documents\workspace\solar-project\Data\monthly_insolation_2023.11-2024.12.csv

df.columns = ["latitude", "longitude","insolation per day (kmh/m^2)"]
df

,latitude,longitude,insolation per day (kmh/m^2)
0,38.4600,127.6984,2.7395
1,38.4599,127.7043,2.7264
2,38.4598,127.7101,2.7317
3,38.4597,127.7160,2.7680
4,38.4596,127.7219,2.8127
...,...,...,...
1244245,33.1046,126.7130,4.9290
1244246,33.1046,126.7185,4.9534
1244247,33.1046,126.7239,4.9583
1244248,33.1045,126.7293,4.9630


In [56]:
#일평균 일사량을 월 합계 일사량으로 바꾼 column 추가하기

df["insolation per month (kmh/m^2)"] = df["insolation per day (kmh/m^2)"] * 31
df

,latitude,longitude,insolation per day (kmh/m^2),insolation per month (kmh/m^2)
0,38.4600,127.6984,2.7395,84.9245
1,38.4599,127.7043,2.7264,84.5184
2,38.4598,127.7101,2.7317,84.6827
3,38.4597,127.7160,2.7680,85.8080
4,38.4596,127.7219,2.8127,87.1937
...,...,...,...,...
1244245,33.1046,126.7130,4.9290,152.7990
1244246,33.1046,126.7185,4.9534,153.5554
1244247,33.1046,126.7239,4.9583,153.7073
1244248,33.1045,126.7293,4.9630,153.8530


좌표 도시별 분류 및 도시별 월 평균 합계 일사량 값 구하기

In [57]:
import geopandas as gpd
from shapely.geometry import Point

#지도 생성

m = folium.Map(location=[36.5, 127.5], zoom_start=7)

#각 도시를 색상으로 표시
#행정동 경계 데이터 파일 링크 (https://github.com/vuski/admdongkor/blob/master/ver20241001/HangJeongDong_ver20241001.geojson)

#관측 지점의 좌표가 Geojson 데이터의 어느 도시에 속하는지 확인하는 코드
geojson_path = "..\\Data\\HangJeongDong_ver20241001.geojson"

gdf = gpd.read_file(geojson_path)

#확인할 좌표 리스트
df['coord'] = list(zip(df['latitude'], df["longitude"]))
df

,latitude,longitude,insolation per day (kmh/m^2),insolation per month (kmh/m^2),coord
0,38.4600,127.6984,2.7395,84.9245,"(38.46, 127.6984)"
1,38.4599,127.7043,2.7264,84.5184,"(38.4599, 127.7043)"
2,38.4598,127.7101,2.7317,84.6827,"(38.4598, 127.7101)"
3,38.4597,127.7160,2.7680,85.8080,"(38.4597, 127.716)"
4,38.4596,127.7219,2.8127,87.1937,"(38.4596, 127.7219)"
...,...,...,...,...,...
1244245,33.1046,126.7130,4.9290,152.7990,"(33.1046, 126.713)"
1244246,33.1046,126.7185,4.9534,153.5554,"(33.1046, 126.7185)"
1244247,33.1046,126.7239,4.9583,153.7073,"(33.1046, 126.7239)"
1244248,33.1045,126.7293,4.9630,153.8530,"(33.1045, 126.7293)"


In [58]:
#각 좌표에 대한 확인
results = []

for i in df["coord"]:
    point = Point(i[1], i[0]) #Geojson은 (경도, 위도) 순서니까!
    match = gdf[gdf.contains(point)] #좌표 지점을 포함하는 폴리곤(도시) 검색

    if not match.empty: #매치되는 도시가 있다면
        city_name = match.iloc[0]["adm_nm"]
        words = city_name.split()
        results.append(words[0]+" "+words[1]) #시군구 기준 표기로 통일
    else: 
        results.append("해당 도시 없음")

df["city"] = results

df #코드 실행하는 데 50분 걸림....;;에휴 이거 다음엔 파일로 저장하든가 해야지

,latitude,longitude,insolation per day (kmh/m^2),insolation per month (kmh/m^2),coord,city
0,38.4600,127.6984,2.7395,84.9245,"(38.46, 127.6984)",해당 도시 없음
1,38.4599,127.7043,2.7264,84.5184,"(38.4599, 127.7043)",해당 도시 없음
2,38.4598,127.7101,2.7317,84.6827,"(38.4598, 127.7101)",해당 도시 없음
3,38.4597,127.7160,2.7680,85.8080,"(38.4597, 127.716)",해당 도시 없음
4,38.4596,127.7219,2.8127,87.1937,"(38.4596, 127.7219)",해당 도시 없음
...,...,...,...,...,...,...
1244245,33.1046,126.7130,4.9290,152.7990,"(33.1046, 126.713)",해당 도시 없음
1244246,33.1046,126.7185,4.9534,153.5554,"(33.1046, 126.7185)",해당 도시 없음
1244247,33.1046,126.7239,4.9583,153.7073,"(33.1046, 126.7239)",해당 도시 없음
1244248,33.1045,126.7293,4.9630,153.8530,"(33.1045, 126.7293)",해당 도시 없음


In [59]:
#해당 도시 없는 좌표의 데이터들을 삭제 (바다, 북한 등)

df = df[df["city"] != "해당 도시 없음"]
#df.to_csv("2020-08_insol_filtered.csv")
df

,latitude,longitude,insolation per day (kmh/m^2),insolation per month (kmh/m^2),coord,city
101,38.4484,128.2927,2.9425,91.2175,"(38.4484, 128.2927)",강원특별자치도 고성군
102,38.4483,128.2986,2.9376,91.0656,"(38.4483, 128.2986)",강원특별자치도 고성군
103,38.4481,128.3044,2.9286,90.7866,"(38.4481, 128.3044)",강원특별자치도 고성군
104,38.4480,128.3103,2.9194,90.5014,"(38.448, 128.3103)",강원특별자치도 고성군
105,38.4479,128.3162,2.9195,90.5045,"(38.4479, 128.3162)",강원특별자치도 고성군
...,...,...,...,...,...,...
1228842,33.1708,126.2741,4.8941,151.7171,"(33.1708, 126.2741)",제주특별자치도 서귀포시
1229935,33.1663,126.2686,4.9142,152.3402,"(33.1663, 126.2686)",제주특별자치도 서귀포시
1229936,33.1663,126.2741,4.9096,152.1976,"(33.1663, 126.2741)",제주특별자치도 서귀포시
1240878,33.1206,126.2685,5.0999,158.0969,"(33.1206, 126.2685)",제주특별자치도 서귀포시


도시(시군구) 명을 기준으로 8월 합계 일사량 평균값 산출하기

In [60]:
#df에서 "insolation per month (kmh/m^2)" 행과 "city" 행만 남기기

insol_df = df[["city", "insolation per month (kmh/m^2)"]]
insol_df = insol_df.reset_index(drop=True)
insol_df


,city,insolation per month (kmh/m^2)
0,강원특별자치도 고성군,91.2175
1,강원특별자치도 고성군,91.0656
2,강원특별자치도 고성군,90.7866
3,강원특별자치도 고성군,90.5014
4,강원특별자치도 고성군,90.5045
...,...,...
386022,제주특별자치도 서귀포시,151.7171
386023,제주특별자치도 서귀포시,152.3402
386024,제주특별자치도 서귀포시,152.1976
386025,제주특별자치도 서귀포시,158.0969


In [61]:
#같은 도시에 있는 8월 합계 일사량끼리 평균하기


city_insol_df = insol_df.groupby("city")["insolation per month (kmh/m^2)"].mean().reset_index()
city_insol_df #도시별 모든 좌표에서 평균한 8월 합계 일사량 데이터 프레임 산출

,city,insolation per month (kmh/m^2)
0,강원특별자치도 강릉시,95.429758
1,강원특별자치도 고성군,87.660571
2,강원특별자치도 동해시,98.332717
3,강원특별자치도 삼척시,102.348656
4,강원특별자치도 속초시,85.784591
...,...,...
246,충청북도 청주시상당구,113.910314
247,충청북도 청주시서원구,116.755406
248,충청북도 청주시청원구,113.541940
249,충청북도 청주시흥덕구,115.519612


2020년 8월의 월 합계 입사량 데이터를 이용하여 도시별 색칠하기

In [62]:
from branca.colormap import linear
from matplotlib.colors import Normalize, rgb2hex
from matplotlib import colormaps 

#일사량을 컬러맵으로 매핑

min_insol = city_insol_df['insolation per month (kmh/m^2)'].min()
max_insol = city_insol_df['insolation per month (kmh/m^2)'].max()
colormap = colormaps["coolwarm"]



norm = Normalize(vmin = min_insol, vmax = max_insol)
print(min_insol, max_insol) #최소, 최대 일사량 값 확인

city_insol_df["Color(hex)"] = city_insol_df["insolation per month (kmh/m^2)"].apply(lambda insol: rgb2hex(colormap(norm(insol)))) 
# 일사량 값을 정규화 하여 컬러맵으로 변환

city_insol_df

84.6971832372839 142.55226000000002


,city,insolation per month (kmh/m^2),Color(hex)
0,강원특별자치도 강릉시,95.429758,#7699f6
1,강원특별자치도 고성군,87.660571,#4a63d3
2,강원특별자치도 동해시,98.332717,#88abfd
3,강원특별자치도 삼척시,102.348656,#a1c0ff
4,강원특별자치도 속초시,85.784591,#3f53c6
...,...,...,...
246,충청북도 청주시상당구,113.910314,#dedcdb
247,충청북도 청주시서원구,116.755406,#ead5c9
248,충청북도 청주시청원구,113.541940,#dcdddd
249,충청북도 청주시흥덕구,115.519612,#e5d8d1


In [63]:
#GeoJSON 데이터 읍, 면, 동 삭제하고 시 단위로 이름 단순화

def simplify_city_name(full_name):
    if " " in full_name:
        return full_name.split()[0] + " " + full_name.split()[1]
    return full_name

gdf['simple_name'] = gdf["adm_nm"].apply(simplify_city_name) #시 단위로 이름 단순화

#GeoJSON 데이터를 시 단위로 묶기

gdf_grouped = gdf.dissolve(by = "simple_name", as_index=False)
gdf_grouped


,simple_name,geometry,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd
0,강원특별자치도 강릉시,"POLYGON ((128.88276 37.59205, 128.87959 37.589...",강원특별자치도 강릉시 주문진읍,5115025000,51150,51,강원특별자치도,강릉시,32030110
1,강원특별자치도 고성군,"POLYGON ((128.56537 38.27518, 128.56881 38.268...",강원특별자치도 고성군 간성읍,5182025000,51820,51,강원특별자치도,고성군,32600110
2,강원특별자치도 동해시,"POLYGON ((129.15502 37.47399, 129.15486 37.473...",강원특별자치도 동해시 천곡동,5117051000,51170,51,강원특별자치도,동해시,32040510
3,강원특별자치도 삼척시,"POLYGON ((129.08496 37.16004, 129.08486 37.160...",강원특별자치도 삼척시 도계읍,5123025000,51230,51,강원특별자치도,삼척시,32070110
4,강원특별자치도 속초시,"POLYGON ((128.60753 38.1859, 128.60928 38.1836...",강원특별자치도 속초시 영랑동,5121051000,51210,51,강원특별자치도,속초시,32060510
...,...,...,...,...,...,...,...,...,...
247,충청북도 청주시상당구,"POLYGON ((127.56547 36.52933, 127.56594 36.528...",충청북도 청주시상당구 낭성면,4311131000,43111,43,충청북도,청주시상당구,33041310
248,충청북도 청주시서원구,"POLYGON ((127.4806 36.47777, 127.4792 36.47746...",충청북도 청주시서원구 남이면,4311231000,43112,43,충청북도,청주시서원구,33042310
249,충청북도 청주시청원구,"POLYGON ((127.50183 36.64782, 127.50091 36.647...",충청북도 청주시청원구 내수읍,4311425000,43114,43,충청북도,청주시청원구,33044110
250,충청북도 청주시흥덕구,"POLYGON ((127.38844 36.58343, 127.38823 36.583...",충청북도 청주시흥덕구 오송읍,4311325000,43113,43,충청북도,청주시흥덕구,33043110


In [64]:
#Folium에 사용할 데이터 준비: gdf_grouped데이터와 city_insol_df 병합

merged = gdf_grouped.merge(city_insol_df, left_on="simple_name", right_on="city", how = "left")
#print(aug_df.info())
print(merged.info())
merged

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   simple_name                     252 non-null    object  
 1   geometry                        252 non-null    geometry
 2   adm_nm                          252 non-null    object  
 3   adm_cd2                         252 non-null    object  
 4   sgg                             252 non-null    object  
 5   sido                            252 non-null    object  
 6   sidonm                          252 non-null    object  
 7   sggnm                           252 non-null    object  
 8   adm_cd                          252 non-null    object  
 9   city                            251 non-null    object  
 10  insolation per month (kmh/m^2)  251 non-null    float64 
 11  Color(hex)                      251 non-null    object  
dtypes: float64(1),

,simple_name,geometry,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd,city,insolation per month (kmh/m^2),Color(hex)
0,강원특별자치도 강릉시,"POLYGON ((128.88276 37.59205, 128.87959 37.589...",강원특별자치도 강릉시 주문진읍,5115025000,51150,51,강원특별자치도,강릉시,32030110,강원특별자치도 강릉시,95.429758,#7699f6
1,강원특별자치도 고성군,"POLYGON ((128.56537 38.27518, 128.56881 38.268...",강원특별자치도 고성군 간성읍,5182025000,51820,51,강원특별자치도,고성군,32600110,강원특별자치도 고성군,87.660571,#4a63d3
2,강원특별자치도 동해시,"POLYGON ((129.15502 37.47399, 129.15486 37.473...",강원특별자치도 동해시 천곡동,5117051000,51170,51,강원특별자치도,동해시,32040510,강원특별자치도 동해시,98.332717,#88abfd
3,강원특별자치도 삼척시,"POLYGON ((129.08496 37.16004, 129.08486 37.160...",강원특별자치도 삼척시 도계읍,5123025000,51230,51,강원특별자치도,삼척시,32070110,강원특별자치도 삼척시,102.348656,#a1c0ff
4,강원특별자치도 속초시,"POLYGON ((128.60753 38.1859, 128.60928 38.1836...",강원특별자치도 속초시 영랑동,5121051000,51210,51,강원특별자치도,속초시,32060510,강원특별자치도 속초시,85.784591,#3f53c6
...,...,...,...,...,...,...,...,...,...,...,...,...
247,충청북도 청주시상당구,"POLYGON ((127.56547 36.52933, 127.56594 36.528...",충청북도 청주시상당구 낭성면,4311131000,43111,43,충청북도,청주시상당구,33041310,충청북도 청주시상당구,113.910314,#dedcdb
248,충청북도 청주시서원구,"POLYGON ((127.4806 36.47777, 127.4792 36.47746...",충청북도 청주시서원구 남이면,4311231000,43112,43,충청북도,청주시서원구,33042310,충청북도 청주시서원구,116.755406,#ead5c9
249,충청북도 청주시청원구,"POLYGON ((127.50183 36.64782, 127.50091 36.647...",충청북도 청주시청원구 내수읍,4311425000,43114,43,충청북도,청주시청원구,33044110,충청북도 청주시청원구,113.541940,#dcdddd
250,충청북도 청주시흥덕구,"POLYGON ((127.38844 36.58343, 127.38823 36.583...",충청북도 청주시흥덕구 오송읍,4311325000,43113,43,충청북도,청주시흥덕구,33043110,충청북도 청주시흥덕구,115.519612,#e5d8d1


In [65]:
#GeoJSON 스타일 함수 정의

def style_function(feature):
    color = feature['properties'].get('Color(hex)', None)
    if color: #색상이 있는 경우 
        return {
            "fillColor" : color,
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0.7
        }
    else: #색상 데이터가 없는 경우 투명 처리
        return {
            "fillColor" : "transparent",
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0
        }

#GeoJSON 레이어 추가
folium.GeoJson(
    merged,
    style_function = style_function,
    tooltip = folium.GeoJsonTooltip(fields=["simple_name", "insolation per month (kmh/m^2)"])
).add_to(m)

In [66]:
m.save("..\\Visualization\\insolation_map(2020-08).html")
print("지도가 'insolation_map(2020-08).html'로 저장되었습니다. 브라우저에서 확인하세요.")

지도가 'insolation_map(2020-08).html'로 저장되었습니다. 브라우저에서 확인하세요.
